In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import qgrid
from numpy.random import normal, seed

import warnings
warnings.filterwarnings('ignore')

import math

import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error


ModuleNotFoundError: No module named 'xgboost'

In [2]:
import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)
    
else:
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False

# 전처리

## 데이터 불러오기

### 날씨

In [57]:
weather_2013 = pd.read_csv('./weather_20130101_20131231.csv', encoding='cp949')
weather_2014 = pd.read_csv('./weather_20140101_20141231.csv', encoding='cp949')
weather_2015 = pd.read_csv('./weather_20150101_20151231.csv', encoding='cp949')
weather_2016 = pd.read_csv('./weather_20160101_20161231.csv', encoding='cp949')
weather_2017 = pd.read_csv('./weather_20170101_20171231.csv', encoding='cp949')
weather_2018 = pd.read_csv('./weather_20180101_20181231.csv', encoding='cp949')
weather = pd.concat([weather_2013, weather_2014, weather_2015, weather_2016, weather_2017, weather_2018], axis=0)

In [58]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4938688 entries, 0 to 831899
Data columns (total 38 columns):
 #   Column         Dtype  
---  ------         -----  
 0   지점             int64  
 1   지점명            object 
 2   일시             object 
 3   기온(°C)         float64
 4   기온 QC플래그       float64
 5   강수량(mm)        float64
 6   강수량 QC플래그      float64
 7   풍속(m/s)        float64
 8   풍속 QC플래그       float64
 9   풍향(16방위)       float64
 10  풍향 QC플래그       float64
 11  습도(%)          float64
 12  습도 QC플래그       float64
 13  증기압(hPa)       float64
 14  이슬점온도(°C)      float64
 15  현지기압(hPa)      float64
 16  현지기압 QC플래그     float64
 17  해면기압(hPa)      float64
 18  해면기압 QC플래그     float64
 19  일조(hr)         float64
 20  일조 QC플래그       float64
 21  일사(MJ/m2)      float64
 22  일사 QC플래그       float64
 23  적설(cm)         float64
 24  3시간신적설(cm)     float64
 25  전운량(10분위)      float64
 26  중하층운량(10분위)    float64
 27  운형(운형약어)       object 
 28  최저운고(100m )    float64
 29  시정(10m)        

In [59]:
weather.head()

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),지면온도 QC플래그,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,90,속초,2013-01-01 00:00,-4.9,0.0,NaN,NaN,6.1,0.0,250.0,...,10.0,2000.0,NaN,NaN,-4.9,0.0,NaN,NaN,NaN,NaN
1,90,속초,2013-01-01 01:00,-4.6,0.0,NaN,NaN,5.5,0.0,250.0,...,NaN,NaN,NaN,NaN,-4.7,0.0,NaN,NaN,NaN,NaN
2,90,속초,2013-01-01 02:00,-4.4,0.0,NaN,NaN,4.3,0.0,270.0,...,NaN,NaN,NaN,NaN,-4.7,0.0,NaN,NaN,NaN,NaN
3,90,속초,2013-01-01 03:00,-5.0,0.0,NaN,NaN,3.5,0.0,250.0,...,NaN,2000.0,4.0,NaN,-4.9,0.0,NaN,NaN,NaN,NaN
4,90,속초,2013-01-01 04:00,-5.9,0.0,NaN,NaN,2.7,0.0,290.0,...,NaN,2000.0,NaN,NaN,-5.6,0.0,NaN,NaN,NaN,NaN


In [60]:
weather = weather.fillna(0)

### 가스 공급량

In [28]:
data = pd.read_csv('./data.csv', encoding="CP949")

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   연월일     368088 non-null  object 
 1   시간      368088 non-null  int64  
 2   구분      368088 non-null  object 
 3   공급량     368088 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 11.2+ MB


In [31]:
data['연월일'] = pd.to_datetime(data['연월일'])

In [32]:
data['time'] = data.apply(lambda x: x['연월일'] + timedelta(hours=x['시간']), axis=1)

In [33]:
data

,연월일,시간,구분,공급량,time
0,2013-01-01,1,A,2497.129,2013-01-01 01:00:00
1,2013-01-01,2,A,2363.265,2013-01-01 02:00:00
2,2013-01-01,3,A,2258.505,2013-01-01 03:00:00
3,2013-01-01,4,A,2243.969,2013-01-01 04:00:00
4,2013-01-01,5,A,2344.105,2013-01-01 05:00:00
...,...,...,...,...,...
368083,2018-12-31,20,H,681.033,2018-12-31 20:00:00
368084,2018-12-31,21,H,669.961,2018-12-31 21:00:00
368085,2018-12-31,22,H,657.941,2018-12-31 22:00:00
368086,2018-12-31,23,H,610.953,2018-12-31 23:00:00


In [34]:
data = data.drop(['연월일', '시간'], axis=1)

In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   구분      368088 non-null  object        
 1   공급량     368088 non-null  float64       
 2   time    368088 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 8.4+ MB


## 데이터 클랜징

In [61]:
weather = weather.drop(['기온 QC플래그', '강수량 QC플래그', '풍속 QC플래그', '풍향(16방위)', 
                   '풍향 QC플래그', '습도(%)', '습도 QC플래그', '증기압(hPa)', '이슬점온도(°C)',
                   '현지기압(hPa)', '현지기압 QC플래그', '해면기압(hPa)', '해면기압 QC플래그',
                   '일조 QC플래그', '일조(hr)', '일사 QC플래그', '3시간신적설(cm)', '전운량(10분위)',
                   '중하층운량(10분위)', '운형(운형약어)', '최저운고(100m )', '시정(10m)',
                   '지면상태(지면상태코드)', '현상번호(국내식)', '지면온도 QC플래그', '5cm 지중온도(°C)',
                   '10cm 지중온도(°C)', '10cm 지중온도(°C)', '30cm 지중온도(°C)'], axis=1)

In [66]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52581 entries, 87593 to 105097
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   지점             52581 non-null  int64  
 1   지점명            52581 non-null  object 
 2   일시             52581 non-null  object 
 3   기온(°C)         52581 non-null  float64
 4   강수량(mm)        52581 non-null  float64
 5   풍속(m/s)        52581 non-null  float64
 6   일사(MJ/m2)      52581 non-null  float64
 7   적설(cm)         52581 non-null  float64
 8   지면온도(°C)       52581 non-null  float64
 9   20cm 지중온도(°C)  52581 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 4.4+ MB


In [63]:
weather['지점명'].unique()

array(['속초', '철원', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉', '강릉', '동해',
       '서울', '인천', '원주', '울릉도', '수원', '영월', '충주', '서산', '울진', '청주', '대전',
       '추풍령', '안동', '상주', '포항', '군산', '대구', '전주', '울산', '창원', '광주', '부산',
       '통영', '목포', '여수', '흑산도', '완도', '고창', '순천', '진도(첨찰산)', '대구(기)',
       '제주', '고산', '성산', '서귀포', '진주', '강화', '양평', '이천', '인제', '홍천', '태백',
       '정선군', '제천', '보은', '천안', '보령', '부여', '금산', '부안', '임실', '정읍', '남원',
       '장수', '고창군', '영광군', '김해시', '순창군', '북창원', '양산시', '보성군', '강진군', '장흥',
       '해남', '고흥', '의령군', '함양군', '광양시', '봉화', '영주', '문경', '청송군', '영덕',
       '의성', '구미', '영천', '경주시', '거창', '합천', '밀양', '산청', '거제', '남해', '진도군',
       '관악산', '홍성', '북춘천'], dtype=object)

In [65]:
weather = weather[weather['지점명']=='서울']

In [29]:
data['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [67]:
weather['time']=pd.to_datetime(weather['일시'])

In [68]:
df = pd.DataFrame()
df = pd.merge(data, weather, on='time')

## 데이터 정규화

In [70]:
from sklearn.preprocessing import MinMaxScaler

In [71]:
d_map = {}
for i, d in enumerate(data['구분'].unique()):
    d_map[d] = i
    
data['구분'] = data['구분'].map(d_map)

In [84]:
scaler = MinMaxScaler()
scaled_cols = ['공급량']
data_scaled = scaler.fit_transform(df[scaled_cols])
df['scaled_공급량'] = data_scaled

## EDA 

In [75]:
# hour, dayofweek, quarter, month, tear, dayofyear, day, weeokofyear로 나누기

df['hour'] = df['time'].dt.hour
df['dayofweek'] = df['time'].dt.dayofweek
df['quarter'] = df['time'].dt.quarter
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year
df['dayofyear'] = df['time'].dt.dayofyear
df['dayofmonth'] = df['time'].dt.day
df['weekofyear'] = df['time'].dt.weekofyear

## train, val 나누기

In [85]:
train_year = [2013, 2014, 2015, 2016, 2017]
val_year = [2018]

In [86]:
train = df[df['year'].isin(train_year)]
val = df[df['year'].isin(val_year)]

In [89]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306747 entries, 0 to 306746
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   구분             306747 non-null  object        
 1   공급량            306747 non-null  float64       
 2   time           306747 non-null  datetime64[ns]
 3   지점             306747 non-null  int64         
 4   지점명            306747 non-null  object        
 5   일시             306747 non-null  object        
 6   기온(°C)         306747 non-null  float64       
 7   강수량(mm)        306747 non-null  float64       
 8   풍속(m/s)        306747 non-null  float64       
 9   일사(MJ/m2)      306747 non-null  float64       
 10  적설(cm)         306747 non-null  float64       
 11  지면온도(°C)       306747 non-null  float64       
 12  20cm 지중온도(°C)  306747 non-null  float64       
 13  hour           306747 non-null  int64         
 14  dayofweek      306747 non-null  int64         
 15  

In [97]:
train_y = train[['scaled_공급량']]
train_x = train.drop(['scaled_공급량'], axis=1)

val_y = val[['scaled_공급량']]
val_x = val.drop(['scaled_공급량'], axis=1)

In [98]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

((306747, 21), (306747, 1), (61313, 21), (61313, 1))

# EDA

# 학습

## XGboost

In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(train_x, train_y,
        eval_set=[(train_x, train_y), (val_x, val_y)],
        early_stopping_rounds=50,
       verbose=False)

In [ ]:
_ = plot_importance(reg, height=0.9)

In [ ]:
df['scaled_공급량'] = red.predict(val_x)
df = pd.conca['']

In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model


from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequentail()
model.add(LSTM(16,
              input_shape=(train_x.shape[1], train_x.shape[2]),
              activation='relu',
              return_squences=False)
         )
model.add(Dense(1))

AlreadyExistsError: Another metric with the same name already exists.

In [134]:
import lightgbm as lgb

In [135]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective':'regression',
    'metric':'mae',
    'seed':42
}


In [136]:
model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

C:\Users\SeunghaLee\.conda\envs\data\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\SeunghaLee\.conda\envs\data\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 244.857
[40]	valid_0's l1: 174.855
[60]	valid_0's l1: 158.739
[80]	valid_0's l1: 153.323
[100]	valid_0's l1: 150.948
[120]	valid_0's l1: 150.463
Early stopping, best iteration is:
[112]	valid_0's l1: 150.297


# 추론 및 결과 제출

In [22]:
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')

In [23]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [24]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [27]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [29]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [30]:
test['구분'] = test['구분'].map(d_map)

In [31]:
test_x = test[features]

In [32]:
test_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [33]:
preds = model.predict(test_x)

In [34]:
submission['공급량'] = preds

In [38]:
submission.to_csv('baseline.csv', index=False)